<a href="https://colab.research.google.com/github/matthewpecsok/data_engineering/blob/main/tutorials/de_tutorial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install --upgrade ipython
!pip -q install --upgrade ipython-sql

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
!wget -O movies.db https://github.com/matthewpecsok/data_engineering/blob/main/data/movies.sqlite?raw=true

con = sqlite3.connect('movies.db')

%load_ext sql
%sql sqlite:///movies.db

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
%%sql SELECT name, sql FROM sqlite_master
WHERE type='table'
ORDER BY name;

In [ ]:
%sql select count(1) from movies

In [ ]:
%sql select count(1) as director_count from directors

In [ ]:
%sql select * from movies limit 5

In [ ]:
%sql select * from directors limit 5

In [ ]:
%sql select max(budget),max(revenue) from movies

In [ ]:
%sql select budget,title from movies order by budget desc limit 10

In [ ]:
%sql select min(release_date),max(release_date),max(release_date) - min(release_date) as years_of_releases from movies

In [ ]:
%%sql select

avg(budget),
avg(revenue),
director_id

from movies

group by director_id order by (avg(budget)) desc
limit 5

In [ ]:
%sql select distinct(department) from directors

In [ ]:
%%sql select *

from movies m
left join directors d on m.director_id = d.id


limit 80

In [ ]:
%%sql select

avg(budget),
avg(revenue),
director_id,
name,
count(1) as movie_count,
max(m.title)


from movies m
left join directors d on m.director_id = d.id

group by director_id
order by avg(budget)
desc
limit 5


In [ ]:
%sql select title from movies where title like '%Star Wars%'